In [ ]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.scenes import add_cloth_competition_dual_ur5e_scene, X_W_LCB_DEFAULT, X_W_RCB_DEFAULT
from airo_drake import X_URBASE_ROSBASE


robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)

X_W_LCB = X_W_LCB_DEFAULT
X_W_RCB = X_W_RCB_DEFAULT

(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_LCB, X_W_RCB
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
home_joints_left = np.deg2rad([180, -120, 60, -30, -90, -90])
home_joints_right = np.deg2rad([-180, -60, -60, -150, 90, 90])
home_joints = np.concatenate([home_joints_left, home_joints_right])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, home_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization


In [ ]:
from functools import partial
from cloth_tools.kinematics.constants import TCP_TRANSFORM
from cloth_tools.kinematics.inverse_kinematics import inverse_kinematics_in_world_fn

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

In [ ]:
from airo_typing import Vector3DType
from airo_drake import visualize_frame
from pydrake.math import RotationMatrix
from pydrake.trajectories import PiecewisePose
from airo_typing import HomogeneousMatrixType
from airo_drake import discretize_drake_pose_trajectory

def make_grasp_pose_vertical(gripper_open_direction: Vector3DType) -> HomogeneousMatrixType:
    grasp_location = np.array([0.2, 0.0, 0.7])

    gripper_forward_direction = np.array([0, 0, 1])
    Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
    # X = np.array([0, -1, 0])
    X = gripper_open_direction / np.linalg.norm(gripper_open_direction)
    # NOTE: we assume that the given X is perpendicular to Z

    Y = np.cross(Z, X)

    grasp_orientation = np.column_stack([X, Y, Z])
    grasp_pose_vertical = np.identity(4)
    grasp_pose_vertical[0:3, 0:3] = grasp_orientation
    grasp_pose_vertical[0:3, 3] = grasp_location
    return grasp_pose_vertical

grasp_pose_paths = []
pose_names = []

for angle in np.linspace(0, np.pi / 2, 7):

    gripper_open_direction =  np.array([1, 0, 0]) if angle <= np.pi / 4 else np.array([0, -1, 0])
    grasp_pose = make_grasp_pose_vertical(gripper_open_direction)

    global_y_rotation = RotationMatrix.MakeYRotation(angle).matrix()
    grasp_pose[:3, :3] = global_y_rotation @ grasp_pose[:3, :3]

    for distance in np.linspace(0.05, 0.50, 10):
        pregrasp_pose = grasp_pose.copy()
        pregrasp_pose[0:3, 3] -= distance * grasp_pose[0:3, 2]
        
        rigid_transforms = [RigidTransform(pose) for pose in [pregrasp_pose, grasp_pose]]
        times = np.linspace(0, 1, len(rigid_transforms))
        pose_trajectory = PiecewisePose.MakeLinear(times=times, poses=rigid_transforms)
        steps = max(2, int(distance / 0.01))
        grasp_pose_path = discretize_drake_pose_trajectory(pose_trajectory, steps).poses

        grasp_pose_paths.append(grasp_pose_path)
        pose_name = f"pregrasp_poses/pose_{angle}_{distance}"
        pose_names.append(pose_name)

        visualize_frame(scene.meshcat, pose_name, pregrasp_pose, length=0.05)

print(len(grasp_pose_paths))

In [ ]:
from airo_planner import DualArmOmplPlanner

joint_bounds_lower = np.deg2rad([-360, -195, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 15, 160, 360, 360, 360])
joint_bounds = joint_bounds_lower, joint_bounds_upper

planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [ ]:
from cloth_tools.planning.grasp_planning import plan_to_grasp_pose_path

# grasp_pose_path = grasp_pose_paths[-5]
grasp_pose_path = grasp_pose_paths[35]


trajectory = plan_to_grasp_pose_path(
    planner,
    grasp_pose_path,
    home_joints_left,
    home_joints_right,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    collision_checker.CheckConfigCollisionFree,
    plant,
    with_left=True
)

In [ ]:
from airo_drake import animate_dual_joint_trajectory

animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)